# Exploració de dades

Carregar dades

In [1]:
import pickle

with open('./data/data.pickle', 'rb') as file:
    data = pickle.load(file)

In [2]:
data[0].text

'Señorías, antes de nada quiero recordar a quienes ocupan las tribunas de invitados que no pueden hacer manifestaciones de ningún tipo mientras dura la sesión. Se abre la sesión con el único punto en el orden del día, como ustedes saben, relativo al debate sobre la investidura del candidato a la Presidencia del Gobierno. Para ello, por la secretaria primera de la Cámara se va a proceder a la lectura de la propuesta de candidato a la Presidencia del Gobierno.'

## Tokenizer

### Sentences

In [3]:
import nltk

def sentenceTokenizer(text):
    spanish_sentence_tokenizer = nltk.data.load('tokenizers/punkt/spanish.pickle')
    sentences = spanish_sentence_tokenizer.tokenize(text)
    return sentences

sentenceTokenizer(data[0].text)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/spanish.pickle[0m

  Searched in:
    - 'C:\\Users\\JORDI/nltk_data'
    - 'd:\\sources\\tfg\\venv\\nltk_data'
    - 'd:\\sources\\tfg\\venv\\share\\nltk_data'
    - 'd:\\sources\\tfg\\venv\\lib\\nltk_data'
    - 'C:\\Users\\JORDI\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


### Words

Usando Freeling para asi tambien obtener el lema de cada palabra
```
docker build -t freeling ./docker/freeling
docker run -it --rm --name freeling -p 50005:50005 freeling analyze -f es.cfg --server -p 50005
```

PoS Tag: https://freeling-user-manual.readthedocs.io/en/v4.2/tagsets/tagset-es/


In [150]:
import subprocess
import os
import random
import string

def getRandomString(length):
    return ''.join(random.choice(string.ascii_lowercase) for i in range(length))


class FreelingResult:
    def __init__(self, line):
        parts = line.split()
        self.word = parts[0]
        self.lema = parts[1]
        self.postag = parts[2]
        self.probability = parts[3]
        
    def __str__(self):
        return f'{self.word} {self.lema} {self.postag} {self.probability}'
        
        
class Freeling:
    def __init__(self, port = 50005):
        self.port = port
            
    def analyzer(self, text):
        temp = f'{getRandomString(10)}.txt'
        with open(temp, 'w', encoding='utf-8') as f:
            f.write(text)
        command = f'analyzer_client.exe {self.port} < {temp}'
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=None, shell=True)
        out, _ = process.communicate()
        os.remove(temp)
        out = out.decode('utf-8').strip()
        lines = out.splitlines()
        return [FreelingResult(line) for line in lines if line != ""]
        
freeling = Freeling()
words = freeling.analyzer(sentences[0])
[str(f) for f in words]

['Señorías señoría NCFP000 1',
 ', , Fc 1',
 'antes_de antes_de SP 1',
 'nada nada PI0CS00 0.893578',
 'quiero querer VMIP1S0 1',
 'recordar recordar VMN0000 1',
 'a a SP 0.998775',
 'quienes quien PR0CP00 1',
 'ocupan ocupar VMIP3P0 1',
 'las el DA0FP0 0.988184',
 'tribunas tribuna NCFP000 1',
 'de de SP 0.999961',
 'invitados invitado NCMP000 0.822581',
 'que que PR0CN00 0.550139',
 'no no RN 0.999297',
 'pueden poder VMIP3P0 1',
 'hacer hacer VMN0000 1',
 'manifestaciones manifestación NCFP000 1',
 'de de SP 0.999961',
 'ningún ninguno DI0MS0 1',
 'tipo tipo NCMS000 1',
 'mientras mientras CS 0.88835',
 'dura durar VMIP3S0 0.157895',
 'la el DA0FS0 0.98926',
 'sesión sesión NCFS000 1',
 '. . Fp 1']

In [14]:
def getTokens(freelingResult):
    return [f.word for f in freelingResult]

tokens = getTokens(words)
tokens

['Señorías',
 ',',
 'antes_de',
 'nada',
 'quiero',
 'recordar',
 'a',
 'quienes',
 'ocupan',
 'las',
 'tribunas',
 'de',
 'invitados',
 'que',
 'no',
 'pueden',
 'hacer',
 'manifestaciones',
 'de',
 'ningún',
 'tipo',
 'mientras',
 'dura',
 'la',
 'sesión',
 '.']

## Stopwords

Del vector de paraules anterior anam a filtrar aquelles paraulres que no tenen un significat substancial.

In [93]:
from nltk.corpus import stopwords

spanish_stopwords = stopwords.words('spanish')
postagConjunction = 'C'
postagDeterminer = 'D'
postagPronoun = 'P'
postagAdposition = 'S'
postagPunctuation = 'F'

def filterStopwordsViaPosTagCategory(freelingResult, postTagsCategoryExcluded = [postagConjunction, postagDeterminer, postagPronoun, postagAdposition]):
    filtered = filter(lambda f : not f.postag[0] in postTagsCategoryExcluded, freelingResult)
    return list(filtered)

def containsSomeElement(sublist, list):
    for e in sublist:
        if e in list:
            return True
    return False

def filterStopwordsViaList(freelingResult, stopwords = spanish_stopwords):
    filtered = filter(lambda f: not containsSomeElement(f.word.split('_'), stopwords), freelingResult)
    return list(filtered)

def filterStopwords(freelingResult, postTagsCategoryExcluded = [postagConjunction, postagDeterminer, postagPronoun, postagAdposition], stopwords = spanish_stopwords):
    filtered = filterStopwordsViaPosTagCategory(freelingResult, postTagsCategoryExcluded)
    filtered = filterStopwordsViaList(filtered, stopwords)
    return filtered


In [94]:
wordsFilteredSTPostTag = filterStopwordsViaPosTagCategory(words)
[str(f) for f in wordsFilteredSTPostTag]

['Señorías señoría NCFP000 1',
 ', , Fc 1',
 'quiero querer VMIP1S0 1',
 'recordar recordar VMN0000 1',
 'ocupan ocupar VMIP3P0 1',
 'tribunas tribuna NCFP000 1',
 'invitados invitado NCMP000 0.822581',
 'no no RN 0.999297',
 'pueden poder VMIP3P0 1',
 'hacer hacer VMN0000 1',
 'manifestaciones manifestación NCFP000 1',
 'tipo tipo NCMS000 1',
 'dura durar VMIP3S0 0.157895',
 'sesión sesión NCFS000 1',
 '. . Fp 1']

In [95]:
wordsFilteredSTList = filterStopwordsViaList(words)
[str(f) for f in wordsFilteredSTList]

['Señorías señoría NCFP000 1',
 ', , Fc 1',
 'quiero querer VMIP1S0 1',
 'recordar recordar VMN0000 1',
 'ocupan ocupar VMIP3P0 1',
 'tribunas tribuna NCFP000 1',
 'invitados invitado NCMP000 0.822581',
 'pueden poder VMIP3P0 1',
 'hacer hacer VMN0000 1',
 'manifestaciones manifestación NCFP000 1',
 'ningún ninguno DI0MS0 1',
 'tipo tipo NCMS000 1',
 'mientras mientras CS 0.88835',
 'dura durar VMIP3S0 0.157895',
 'sesión sesión NCFS000 1',
 '. . Fp 1']

In [96]:
wordsFilteredST = filterStopwords(words)
[str(f) for f in wordsFilteredST]

['Señorías señoría NCFP000 1',
 ', , Fc 1',
 'quiero querer VMIP1S0 1',
 'recordar recordar VMN0000 1',
 'ocupan ocupar VMIP3P0 1',
 'tribunas tribuna NCFP000 1',
 'invitados invitado NCMP000 0.822581',
 'pueden poder VMIP3P0 1',
 'hacer hacer VMN0000 1',
 'manifestaciones manifestación NCFP000 1',
 'tipo tipo NCMS000 1',
 'dura durar VMIP3S0 0.157895',
 'sesión sesión NCFS000 1',
 '. . Fp 1']

## Synonyms

In [9]:
from nltk.corpus.reader.wordnet import WordNetCorpusReader
import numpy as np
import urllib
from bs4 import BeautifulSoup as soup
import unidecode

wncr = WordNetCorpusReader('./wordnet_spa', None)
cache = {}

def getSynonyms(word):
    wn_synonyms = [ss.name().split('.')[0] for ss in wncr.synsets(word)]
    
    eduacalingo_synonyms = []
    try:
        data = urllib.request.urlopen(f'https://educalingo.com/en/dic-es/{unidecode.unidecode(word)}').read()
        eduacalingo_synonyms = [s.text for s in soup(data, 'html.parser').select('.contenido_sinonimos_antonimos0')[0].select('a')]
    except:
        None
    
    synonyms = np.unique(wn_synonyms + eduacalingo_synonyms)
    return synonyms

def getCacheSynonyms(word):
    word = word.lower()
    if word in cache:
        return cache[word]
    else:
        synonyms = getSynonyms(word)
        cache[word] = synonyms
        return synonyms

getCacheSynonyms('cálido')

array(['abrasador', 'acogedor', 'afectivo', 'afectuoso', 'amable',
       'amigable', 'ardiente', 'ardoroso', 'caliente', 'calinoso',
       'caluroso', 'candente', 'canicular', 'cordial', 'cálido',
       'entrañable', 'moderado', 'sofocante', 'suave', 'templado',
       'tropical', 'tórrido'], dtype='<U10')

## N-Grams

In [79]:
from nltk.util import ngrams

def getNGrams(freelingResult, size = 2):
    listNGrams = list(ngrams(freelingResult, size))
    return listNGrams

twoGrams = getNGrams(words)
[[f[0].word, f[1].word] for f in twoGrams]


[['Señorías', ','],
 [',', 'antes_de'],
 ['antes_de', 'nada'],
 ['nada', 'quiero'],
 ['quiero', 'recordar'],
 ['recordar', 'a'],
 ['a', 'quienes'],
 ['quienes', 'ocupan'],
 ['ocupan', 'las'],
 ['las', 'tribunas'],
 ['tribunas', 'de'],
 ['de', 'invitados'],
 ['invitados', 'que'],
 ['que', 'no'],
 ['no', 'pueden'],
 ['pueden', 'hacer'],
 ['hacer', 'manifestaciones'],
 ['manifestaciones', 'de'],
 ['de', 'ningún'],
 ['ningún', 'tipo'],
 ['tipo', 'mientras'],
 ['mientras', 'dura'],
 ['dura', 'la'],
 ['la', 'sesión'],
 ['sesión', '.']]

Dels *2-grames* anteriors podem notar la presencia de signes de puntuació, per la propia naturalesa d'aquets separen unitats d'informació.

In [80]:
def containsPunctuationMarks(ngramFreeLing):
    for ng in ngramFreeLing:
        if ng.postag.startswith('F'):
            return True
    return False

def getNGramsFilteringPunctuationMarks(freelingResult, size = 2):
    listNGrams = getNGrams(freelingResult, size)    
    filtered = filter(lambda ng: not containsPunctuationMarks(ng), listNGrams)
    return filtered
    
    
filteredTwoGrams = getNGramsFilteringPunctuationMarks(words)
[[f[0].word, f[1].word] for f in filteredTwoGrams]

[['antes_de', 'nada'],
 ['nada', 'quiero'],
 ['quiero', 'recordar'],
 ['recordar', 'a'],
 ['a', 'quienes'],
 ['quienes', 'ocupan'],
 ['ocupan', 'las'],
 ['las', 'tribunas'],
 ['tribunas', 'de'],
 ['de', 'invitados'],
 ['invitados', 'que'],
 ['que', 'no'],
 ['no', 'pueden'],
 ['pueden', 'hacer'],
 ['hacer', 'manifestaciones'],
 ['manifestaciones', 'de'],
 ['de', 'ningún'],
 ['ningún', 'tipo'],
 ['tipo', 'mientras'],
 ['mientras', 'dura'],
 ['dura', 'la'],
 ['la', 'sesión']]

In [97]:
filteredThreeGrams = getNGramsFilteringPunctuationMarks(wordsFilteredST, size=3)
[[f[0].lema, f[1].lema, f[2].lema] for f in filteredThreeGrams]

[['querer', 'recordar', 'ocupar'],
 ['recordar', 'ocupar', 'tribuna'],
 ['ocupar', 'tribuna', 'invitado'],
 ['tribuna', 'invitado', 'poder'],
 ['invitado', 'poder', 'hacer'],
 ['poder', 'hacer', 'manifestación'],
 ['hacer', 'manifestación', 'tipo'],
 ['manifestación', 'tipo', 'durar'],
 ['tipo', 'durar', 'sesión']]

## Counting

In [154]:
debateTitle = data[0].meta['debate']
debate = [d for d in data if d.meta['debate'] == debateTitle]

freelings = []
for d in debate:
    d.meta['freeling'] = Freeling().analyzer(d.text)
    freelings = freelings + d.meta['freeling']

In [173]:
debate[1].text

'Gracias, presidente. De acuerdo con lo establecido en el artículo 99.1 de la Constitución, tras celebrar consultas con los representantes designados por los grupos políticos con representación parlamentaria, vengo en proponer al Excmo. señor don Pedro Sánchez Pérez-Castejón, como candidato a la Presidencia del Gobierno. Lo que comunico a V.E. para que se formule al Congreso de los Diputados la oportuna propuesta. Palacio de la Zarzuela, 2 de febrero de 2016. Felipe VI, Rey. El presidente del Congreso de los Diputados, Patxi López Álvarez.'

In [165]:
from collections import Counter

filteredFreelings = filterStopwordsViaPosTagCategory(freelings, postTagsCategoryExcluded = [postagConjunction, postagDeterminer, postagPronoun, postagAdposition, postagPunctuation])
lemasAndPoS = [f.lema + ' ' + f.postag for f in filteredFreelings]

bagOfWords = Counter(lemasAndPoS)
bagOfWords.most_common(20)

[('no RN', 218),
 ('ser VSIP3S0', 217),
 ('señoría NCFP000', 191),
 ('gobierno NP00O00', 111),
 ('más RG', 92),
 ('acuerdo NCMS000', 73),
 ('españa NP00G00', 70),
 ('cambio NCMS000', 62),
 ('también RG', 60),
 ('español NCMP000', 58),
 ('haber VAIP3S0', 49),
 ('haber VAIP3P0', 48),
 ('año NCMP000', 47),
 ('hoy RG', 43),
 ('aplausos NP00G00', 42),
 ('hacer VMN0000', 38),
 ('diálogo NCMS000', 38),
 ('haber VAIP1P0', 36),
 ('cámara NP00O00', 35),
 ('estado NP00O00', 35)]

In [167]:
uniqueWords = len(bagOfWords)
uniqueWords

3458

In [169]:
bagOfWords['no RN'] / uniqueWords

0.06304222093695779